In [ ]:
import requests
import pandas as pd
import io
import numpy as np
import os
import warnings

#!pip install fuzzywuzzy
import fuzzywuzzy

from fuzzywuzzy import fuzz
import re

warnings.filterwarnings("ignore")
import spacy
nlp = spacy.load("en_core_web_sm")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


/usr/local/lib/python3.8/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
URL = "https://raw.githubusercontent.com/seantrott/raw-c/main/data/processed/raw-c.csv"
download = requests.get(URL).content
df = pd.read_csv(io.StringIO(download.decode('utf-8')),sep = ',')

In [ ]:
df.head()

,word,sentence1,sentence2,same,ambiguity_type,disambiguating_word1,disambiguating_word2,version,Class,mean_relatedness,median_relatedness,diff,count,sd_relatedness,distance_bert,distance_elmo,se_relatedness,v1,v2
0,act,It was a desperate act.,It was a magic act.,False,Polysemy,desperate,magic,M1_a_M2_a,N,2.181818,2.0,0.181818,11,1.328020,0.204110,0.034093,0.400413,M1_a,M2_a
1,act,It was a desperate act.,It was a comedic act.,False,Polysemy,desperate,comedic,M1_a_M2_b,N,2.000000,2.0,0.000000,7,1.290994,0.215616,0.045927,0.487950,M1_a,M2_b
2,act,It was a humane act.,It was a magic act.,False,Polysemy,humane,magic,M1_b_M2_a,N,2.818182,3.0,0.181818,11,0.981650,0.191488,0.042351,0.295979,M1_b,M2_a
3,act,It was a humane act.,It was a comedic act.,False,Polysemy,humane,comedic,M1_b_M2_b,N,2.809524,3.0,0.190476,21,0.928388,0.225272,0.057707,0.202591,M1_b,M2_b
4,act,It was a desperate act.,It was a humane act.,True,Polysemy,desperate,humane,M1_a_M1_b,N,3.900000,4.0,0.100000,10,0.316228,0.167990,0.041440,0.100000,M1_a,M1_b


In [ ]:
df_dash = df[['word', 'sentence1', 'sentence2', "Class"]]

In [ ]:
df1 = df_dash[['word', 'sentence1', 'Class']]
df1.columns =  ['word', 'sent', 'class']
df1["index"] = df1.groupby("word").cumcount()*2

In [ ]:
df2 =  df_dash[['word', 'sentence2', 'Class']]
df2.columns =  ['word', 'sent', 'class']
df2["index"] = df2.groupby("word").cumcount()*2+1

In [ ]:
df_final = pd.concat([df1, df2])
df_final = df_final.sort_values(by = ['word', 'index'], ascending = [True, True])

In [ ]:
df_final["identifier"] = df_final["word"]+"-"+df_final["index"].astype(str)

In [ ]:
df_odd = df_final[1::2]
df_odd.columns = ["word2", "sent2", 'class2', "index2", "identifier2"]
df_even = df_final[::2]
df_even.columns = ["word1", "sent1", "class1", "index1", "identifier1"]
df_final = pd.concat([df_even, df_odd], axis=1)

In [ ]:
df_final = df_final[["word2", "sent2", "identifier2", 'class2', "sent1", "identifier1", 'class1', 'index1', 'index2']]
df_final_next_stage = df_final.rename(columns={"word2":"lemma"})

In [ ]:
df_final = df_final_next_stage

In [ ]:
df_final_next_stage

,lemma,sent2,identifier2,class2,sent1,identifier1,class1,index1,index2
0,act,It was a magic act.,act-1,N,It was a desperate act.,act-0,N,0,1
1,act,It was a comedic act.,act-3,N,It was a desperate act.,act-2,N,2,3
2,act,It was a magic act.,act-5,N,It was a humane act.,act-4,N,4,5
3,act,It was a comedic act.,act-7,N,It was a humane act.,act-6,N,6,7
4,act,It was a humane act.,act-9,N,It was a desperate act.,act-8,N,8,9
...,...,...,...,...,...,...,...,...,...
667,yard,They were cluttered yards.,yard-3,N,It was five yards.,yard-2,N,2,3
668,yard,They were big yards.,yard-5,N,It was ten yards.,yard-4,N,4,5
669,yard,They were cluttered yards.,yard-7,N,It was ten yards.,yard-6,N,6,7
670,yard,It was ten yards.,yard-9,N,It was five yards.,yard-8,N,8,9


In [ ]:
df_final["judgment"] = df["same"].values.tolist()

In [ ]:
df_final = df_final[['lemma', 'identifier2', 'identifier1', 'judgment']]

In [ ]:
df_final["annotator"] = " "
df_final["comment"] = df['ambiguity_type']

In [ ]:
df_final = df_final[["identifier1", "identifier2", "annotator", "judgment", "comment", "lemma"]]

In [ ]:
df_final.head()

,identifier1,identifier2,annotator,judgment,comment,lemma
0,act-0,act-1,,False,Polysemy,act
1,act-2,act-3,,False,Polysemy,act
2,act-4,act-5,,False,Polysemy,act
3,act-6,act-7,,False,Polysemy,act
4,act-8,act-9,,True,Polysemy,act


In [ ]:
for i in list(df_final["lemma"].value_counts().index):
    df_temp = df_final[df_final["lemma"]==i]
    numpy_df = df_temp.to_numpy()
    header = list(df_temp.columns)
    numpy_df = np.vstack([header, numpy_df])
    if not os.path.exists(i):
        os.mkdir(i)
    np.savetxt(i+"/judgments.csv", numpy_df,fmt='%s', delimiter='\t')

In [ ]:
df.head()

,word,sentence1,sentence2,same,ambiguity_type,disambiguating_word1,disambiguating_word2,version,Class,mean_relatedness,median_relatedness,diff,count,sd_relatedness,distance_bert,distance_elmo,se_relatedness,v1,v2
0,act,It was a desperate act.,It was a magic act.,False,Polysemy,desperate,magic,M1_a_M2_a,N,2.181818,2.0,0.181818,11,1.328020,0.204110,0.034093,0.400413,M1_a,M2_a
1,act,It was a desperate act.,It was a comedic act.,False,Polysemy,desperate,comedic,M1_a_M2_b,N,2.000000,2.0,0.000000,7,1.290994,0.215616,0.045927,0.487950,M1_a,M2_b
2,act,It was a humane act.,It was a magic act.,False,Polysemy,humane,magic,M1_b_M2_a,N,2.818182,3.0,0.181818,11,0.981650,0.191488,0.042351,0.295979,M1_b,M2_a
3,act,It was a humane act.,It was a comedic act.,False,Polysemy,humane,comedic,M1_b_M2_b,N,2.809524,3.0,0.190476,21,0.928388,0.225272,0.057707,0.202591,M1_b,M2_b
4,act,It was a desperate act.,It was a humane act.,True,Polysemy,desperate,humane,M1_a_M1_b,N,3.900000,4.0,0.100000,10,0.316228,0.167990,0.041440,0.100000,M1_a,M1_b


In [ ]:
df1 = df_final_next_stage[['lemma', 'sent1', 'class1', 'identifier1', 'index1']]
df1.columns =  ['word', 'sent', 'pos', 'identifier', 'index']
df2 = df_final_next_stage[['lemma', 'sent2', 'class2', 'identifier2', 'index2']]
df2.columns =  ['word', 'sent', 'pos', 'identifier', 'index']


In [ ]:
df_final = pd.concat([df1, df2])

In [ ]:
df_final.head()

,word,sent,pos,identifier,index
0,act,It was a desperate act.,N,act-0,0
1,act,It was a desperate act.,N,act-2,2
2,act,It was a humane act.,N,act-4,4
3,act,It was a humane act.,N,act-6,6
4,act,It was a desperate act.,N,act-8,8


In [ ]:
df_final = df_final.reset_index(drop = True)

In [ ]:
from fuzzywuzzy import process
tag = ''
def get_indices_of_tags(sentence, word):
   i = sentence.split(" ")
   tag = process.extractOne(word, i)
   tag = tag[0]
   #print(tag)
   tag = re.sub('\W+', '', tag)
   return(str(sentence.find(tag))+":"+str(sentence.find(tag)+len(tag)))


In [ ]:
def get_target(sentence, word):
   tok = sentence.split(" ")
   tag = process.extractOne(word, tok)
   tag = tag[0]
   #print(tag)
   for i in tok:
     if tag == i:
       ind = tok.index(i)
   return ind

In [ ]:
df_final["indexes_target_token"] = df_final.apply(lambda x: get_indices_of_tags(x.sent, x.word), axis=1)

In [ ]:
df_final['indexes_target_token_tokenized'] = df_final.apply(lambda x: get_target(x.sent, x.word), axis=1)

In [ ]:
df_final.head()

,word,sent,pos,identifier,index,indexes_target_token,indexes_target_token_tokenized
0,act,It was a desperate act.,N,act-0,0,19:22,4
1,act,It was a desperate act.,N,act-2,2,19:22,4
2,act,It was a humane act.,N,act-4,4,16:19,4
3,act,It was a humane act.,N,act-6,6,16:19,4
4,act,It was a desperate act.,N,act-8,8,19:22,4


In [ ]:
def get_indices_of_sent(sentence):
    return "0:"+str(len(sentence))

In [ ]:
df_final["indexes_target_sentence"] = df_final["sent"].apply(get_indices_of_sent)

In [ ]:
df_final = df_final.rename(columns = {"sent":"context", "word":"lemma"})

In [ ]:
df_final['description'] = " "
df_final['date'] = " "
df_final['grouping'] = " "

In [ ]:
df_final = df_final[['lemma', 'pos', 'date', 'grouping', 'identifier', 'description', 'context', 'indexes_target_token', 'indexes_target_sentence', 'indexes_target_token_tokenized', 'index' ]]

In [ ]:
df_final

,lemma,pos,date,grouping,identifier,description,context,indexes_target_token,indexes_target_sentence,indexes_target_token_tokenized,index
0,act,N,,,act-0,,It was a desperate act.,19:22,0:23,4,0
1,act,N,,,act-2,,It was a desperate act.,19:22,0:23,4,2
2,act,N,,,act-4,,It was a humane act.,16:19,0:20,4,4
3,act,N,,,act-6,,It was a humane act.,16:19,0:20,4,6
4,act,N,,,act-8,,It was a desperate act.,19:22,0:23,4,8
...,...,...,...,...,...,...,...,...,...,...,...
1339,yard,N,,,yard-3,,They were cluttered yards.,20:25,0:26,3,3
1340,yard,N,,,yard-5,,They were big yards.,14:19,0:20,3,5
1341,yard,N,,,yard-7,,They were cluttered yards.,20:25,0:26,3,7
1342,yard,N,,,yard-9,,It was ten yards.,11:16,0:17,3,9


In [ ]:
df_final['context_tokenized'] = df_final['context']

In [ ]:
def get_len_tok(sentence):
  return "0:"+str(len(sentence.split(" ")))

In [ ]:
df_final['indexes_target_sentence_tokenized'] = df_final["context_tokenized"].apply(get_len_tok)

In [ ]:
lem = ''
def lemmatizr(sent):
  doc = nlp(sent)
  for i in doc:
    lem += i.lemma_
    return lem

In [ ]:
df_final['context_lemmatized'] = df_final["context_tokenized"].apply(lambda row: " ".join([w.lemma_ for w in nlp(row)]))

In [ ]:
df_final['context_pos'] = df_final["context_tokenized"].apply(lambda row: " ".join([w.pos_ for w in nlp(row)]))

In [ ]:
final_df = df_final[['lemma', 'pos', 'date', 'grouping', 'identifier', 'description', 'context', 'indexes_target_token', 'indexes_target_sentence', 'context_tokenized', 'indexes_target_token_tokenized', 'indexes_target_sentence_tokenized', 'context_lemmatized', 'context_pos', 'index']]

In [ ]:
final_df = df_final.sort_values(by = ['lemma', 'index'], ascending = [True, True]).drop("index", axis=1)

In [ ]:
final_df = final_df.reset_index(drop = True)

In [ ]:
final_df

,lemma,pos,date,grouping,identifier,description,context,indexes_target_token,indexes_target_sentence,indexes_target_token_tokenized,context_tokenized,indexes_target_sentence_tokenized,context_lemmatized,context_pos
0,act,N,,,act-0,,It was a desperate act.,19:22,0:23,4,It was a desperate act.,0:5,it be a desperate act .,PRON AUX DET ADJ NOUN PUNCT
1,act,N,,,act-1,,It was a magic act.,15:18,0:19,4,It was a magic act.,0:5,it be a magic act .,PRON AUX DET ADJ NOUN PUNCT
2,act,N,,,act-2,,It was a desperate act.,19:22,0:23,4,It was a desperate act.,0:5,it be a desperate act .,PRON AUX DET ADJ NOUN PUNCT
3,act,N,,,act-3,,It was a comedic act.,17:20,0:21,4,It was a comedic act.,0:5,it be a comedic act .,PRON AUX DET ADJ NOUN PUNCT
4,act,N,,,act-4,,It was a humane act.,16:19,0:20,4,It was a humane act.,0:5,it be a humane act .,PRON AUX DET ADJ NOUN PUNCT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1339,yard,N,,,yard-7,,They were cluttered yards.,20:25,0:26,3,They were cluttered yards.,0:4,they be cluttered yard .,PRON AUX ADJ NOUN PUNCT
1340,yard,N,,,yard-8,,It was five yards.,12:17,0:18,3,It was five yards.,0:4,it be five yard .,PRON AUX NUM NOUN PUNCT
1341,yard,N,,,yard-9,,It was ten yards.,11:16,0:17,3,It was ten yards.,0:4,it be ten yard .,PRON AUX NUM NOUN PUNCT
1342,yard,N,,,yard-10,,They were big yards.,14:19,0:20,3,They were big yards.,0:4,they be big yard .,PRON AUX ADJ NOUN PUNCT


In [ ]:
final_df1 = final_df.explode("description")

In [ ]:
final_df1

,lemma,pos,date,grouping,identifier,description,context,indexes_target_token,indexes_target_sentence,indexes_target_token_tokenized,context_tokenized,indexes_target_sentence_tokenized,context_lemmatized,context_pos
0,act,N,,,act-0,,It was a desperate act.,19:22,0:23,4,It was a desperate act.,0:5,it be a desperate act .,PRON AUX DET ADJ NOUN PUNCT
1,act,N,,,act-1,,It was a magic act.,15:18,0:19,4,It was a magic act.,0:5,it be a magic act .,PRON AUX DET ADJ NOUN PUNCT
2,act,N,,,act-2,,It was a desperate act.,19:22,0:23,4,It was a desperate act.,0:5,it be a desperate act .,PRON AUX DET ADJ NOUN PUNCT
3,act,N,,,act-3,,It was a comedic act.,17:20,0:21,4,It was a comedic act.,0:5,it be a comedic act .,PRON AUX DET ADJ NOUN PUNCT
4,act,N,,,act-4,,It was a humane act.,16:19,0:20,4,It was a humane act.,0:5,it be a humane act .,PRON AUX DET ADJ NOUN PUNCT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1339,yard,N,,,yard-7,,They were cluttered yards.,20:25,0:26,3,They were cluttered yards.,0:4,they be cluttered yard .,PRON AUX ADJ NOUN PUNCT
1340,yard,N,,,yard-8,,It was five yards.,12:17,0:18,3,It was five yards.,0:4,it be five yard .,PRON AUX NUM NOUN PUNCT
1341,yard,N,,,yard-9,,It was ten yards.,11:16,0:17,3,It was ten yards.,0:4,it be ten yard .,PRON AUX NUM NOUN PUNCT
1342,yard,N,,,yard-10,,They were big yards.,14:19,0:20,3,They were big yards.,0:4,they be big yard .,PRON AUX ADJ NOUN PUNCT


In [ ]:
for i in list(final_df["lemma"].value_counts().index):
    df_temp = final_df[final_df["lemma"]==i]
    numpy_df = df_temp.to_numpy()
    header = list(df_temp.columns)
    numpy_df = np.vstack([header, numpy_df])
    if not os.path.exists(i):
        os.mkdir(i)
    np.savetxt(i+"/uses.csv", numpy_df,fmt='%s', delimiter='\t')